In [ ]:
import nglview as ng
import pandas as pd

In [ ]:
%cd data_urea

In [ ]:
!gmx editconf -f MOL.amb2gmx/UREA_GMX.gro -o box1.gro -box 4 3 4

In [ ]:
!gmx insert-molecules -f box1.gro -ci MOL.amb2gmx/UREA_GMX.gro -o box2.gro -nmol 299 -try 100

In [ ]:
view = ng.show_structure_file("box2.gro")
view

In [ ]:
!cat input/emin-urea.mdp

In [ ]:
!gmx grompp -f input/emin-urea.mdp -c box2.gro -p URE300.top -o em.tpr
!gmx mdrun -deffnm em

In [ ]:
!cat input/nvt-urea.mdp

In [ ]:
!gmx grompp -f input/nvt-urea.mdp -c em.gro -r em.gro -p URE300.top -o nvt.tpr 
!gmx mdrun -ntmpi 1 -v -deffnm nvt

In [ ]:
!echo "Temperature" | gmx energy -f nvt.edr -o temperature.xvg -xvg none

In [ ]:
df = pd.read_csv('temperature.xvg', sep='\s+', header=None, names=['time','Urea'])
ax = df.plot('time')
ax.set_xlabel("Time (ps)")
ax.set_ylabel("Temperature (K)")

In [ ]:
!printf "2\n" | gmx dos -f nvt.trr -s nvt.tpr -g Cv.log -b 50

In [ ]:
!tail -3 Cv.log

In [ ]:
!cat input/npt-urea.mdp

In [ ]:
!gmx grompp -f input/npt-urea.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p URE300.top -o npt.tpr
!gmx mdrun -ntmpi 1 -v -deffnm npt

In [ ]:
!echo "Pressure" | gmx energy -f npt.edr -o pressure.xvg -xvg none

In [ ]:
df = pd.read_csv('pressure.xvg', sep='\s+', header=None, names=['time','Urea'])
ax = df.plot('time')
ax.set_xlabel("Time (ps)")
ax.set_ylabel("Pressure (bar)")

In [ ]:
!echo "Density" | gmx energy -f npt.edr -o density.xvg -xvg none

In [ ]:
df = pd.read_csv('density.xvg', sep='\s+', header=None, names=['time','Urea'])
ax = df.plot('time')
ax.set_xlabel("Time (ps)")
ax.set_ylabel("Density (kg/m^3)")

In [ ]:
!printf "2\n" | gmx dos -f npt.trr -s npt.tpr -g Cp.log -b 150

In [ ]:
!tail -3 Cp.log

In [ ]:
!printf "1\n" | gmx dipoles -f npt.trr -s npt.tpr -corr mol -temp 298 -b 150